<center>
    <h1>Analyzing data and preprocessing</h1>
</center>

# 1. Setup

In [1]:
# Import base librairies
import sys
from pathlib import Path
from PIL import Image

# Import scientific librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Tensorflow
from tensorflow import keras

# Add project directory to kernel paths
sys.path.append("../..")

2023-02-28 20:39:51.287936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Import custom functions
from src.visualization.plot_lib import default_viz
from src.data.file_manager import fetch_full_dataset, check_full_dataset, ProjectInfo

zoidbergInfo = ProjectInfo()

# Set default graphics visualization
%matplotlib inline
default_viz()

# 2. Transform data into a common dataset

Before starting any analysis on the data, we just transform the data folder structure into a more convinient one:
- we remove train, val and set folder as we will split data later
- we provide 3 folders : one by class

This folder is in the interim data folder.

In [3]:
fetch_full_dataset()
print('')
is_ok = check_full_dataset(print_result=True)

Full dataset already created

full dataset : class normal => nb images = 1583
full dataset : class bacteria => nb images = 2780
full dataset : class virus => nb images = 1493
----------------------------------------
raw data     : class bacteria => nb images = 2780
raw data     : class normal => nb images = 1583
raw data     : class virus => nb images = 1493


Then, we will create a DataFrame that contains metadata of each images in orders to analyse those images. The metadata we would like are : 
 - Filename
 - Size of the image
 - Number of channels e.g grayscale, rbg, rgba, so on
 - Label of the image  : normal, virus, bacteria

In [5]:
metadata = []
for img_path in Path(zoidbergInfo.data_dir / "interim" / "full").rglob('*.jpeg'):
    with Image.open(img_path) as img:
        label = img_path.parents[0].name
        width, height = img.size
        channels = len(img.getbands())
        
        metadata.append((img_path.name, width, height, channels, label))

metadata_df = pd.DataFrame(metadata, columns=["image", "width", "height", "channels", "label"])

metadata_df.head()

,image name,width,height,channels,label
0,normal_img_1280.jpeg,1546,1271,1,normal
1,normal_img_0868.jpeg,1504,1174,1,normal
2,normal_img_0491.jpeg,1786,1360,1,normal
3,normal_img_0184.jpeg,1450,1149,1,normal
4,normal_img_0887.jpeg,912,672,1,normal


display 2 images per label